In [26]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [27]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [28]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [29]:
!pip install -q findspark

In [30]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [31]:
import findspark
findspark.init()

In [32]:
findspark.find()

'/content/spark-3.0.1-bin-hadoop2.7'

In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [34]:
spark

In [35]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-02-09 01:48:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.211.152.205, 107.21.11.91, 54.235.211.105, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.211.152.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  77.4MB/s    in 0.2s    

2021-02-09 01:48:30 (77.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://395a0ce9cf2a.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5

In [36]:
import io
import csv
import pandas as pd
from pyspark import *
from pyspark import SparkContext as sc, SparkConf
from pyspark.python.pyspark.shell import spark
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
from datetime import datetime
from pyspark.sql.functions import udf, to_date, to_utc_timestamp, lit, col
from pyspark.sql.types import StringType, DateType

In [37]:
from pyspark import SparkContext
import pyspark.sql.functions
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql import SparkSession

In [38]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=  SparkSession.builder.master("local[*]").getOrCreate()

In [46]:
from google.colab import files
files.upload()

Saving passengers_1k.csv to passengers_1k.csv


{'passengers_1k.csv': b'first_name,middle_name,last_name,gender,birthday,email\nPatricio,,Manzanares,M,1953-10-04,patricio.manzanares@gmail.com\nSof\xc3\xada,Lorena,Manzanares,F,1947-11-30,sof\xc3\xada.manzanares@hotmail.com\nJaime,Emanuelly Heather,Tolentino,F,1984-07-10,jaime.tolentino@yahoo.com\nOreste,Ajimin,Tolentino,M,1985-03-19,oreste.tolentino@gmail.com\nAlexandre,John,Tolentino,T,2016-02-05,alexandre.tolentino@yahoo.com\nAlessandra,Leonardo,Anderson,F,1968-05-31,alessandra.anderson@yahoo.com\nMarvin,Joseph,Anderson,M,1966-07-30,marvin.anderson@yahoo.com\nJosep,Michelotto,Anderson,GN,2015-11-17,josep.anderson@gmail.com\nJennifer,Rebekah,Meyer,F,1983-09-05,jennifer.meyer@hotmail.com\nVirginia,Rosita,Meza,F,1996-06-09,virginia.meza@yahoo.com\nHeather,Amanda,Martins,F,1991-09-21,heather.martins@gmail.com\nAldo,Viktor,Douglas,M,2000-02-09,aldo.douglas@hotmail.com\nJonathan,,Jackson,M,2000-05-14,jonathan.jackson@walker.com\nDarryl,Mandy,Schweizer,T,1985-06-17,darryl.schweizer@gmail.

In [56]:
from pyspark.sql import SparkSession
sparksql = SparkSession.builder.master('local[1]').getOrCreate()
df = sparksql.read.csv("passengers_1k.csv",header=True)
df.head()

Row(first_name='Patricio', middle_name=None, last_name='Manzanares', gender='M', birthday='1953-10-04', email='patricio.manzanares@gmail.com')

Transforming the names and standardizing the data 



In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import initcap, col
passengers_df = df.withColumn('first_name', initcap(col('first_name')))\
                             .withColumn('middle_name', initcap(col('middle_name')))\
                             .withColumn('last_name', initcap(col('last_name')))

concatnating name into single column

In [60]:
passengers_df = passengers_df.withColumn('full_name',
                                         concat_ws(" ",
                                                   col('first_name'),
                                                   col('middle_name'),
                                                   col('last_name')))

passengers_df.head(10)

[Row(first_name='Patricio', middle_name=None, last_name='Manzanares', gender='M', birthday='1953-10-04', email='patricio.manzanares@gmail.com', full_name='Patricio Manzanares'),
 Row(first_name='Sofía', middle_name='Lorena', last_name='Manzanares', gender='F', birthday='1947-11-30', email='sofía.manzanares@hotmail.com', full_name='Sofía Lorena Manzanares'),
 Row(first_name='Jaime', middle_name='Emanuelly Heather', last_name='Tolentino', gender='F', birthday='1984-07-10', email='jaime.tolentino@yahoo.com', full_name='Jaime Emanuelly Heather Tolentino'),
 Row(first_name='Oreste', middle_name='Ajimin', last_name='Tolentino', gender='M', birthday='1985-03-19', email='oreste.tolentino@gmail.com', full_name='Oreste Ajimin Tolentino'),
 Row(first_name='Alexandre', middle_name='John', last_name='Tolentino', gender='T', birthday='2016-02-05', email='alexandre.tolentino@yahoo.com', full_name='Alexandre John Tolentino'),
 Row(first_name='Alessandra', middle_name='Leonardo', last_name='Anderson', 

In [63]:
#saving as a parquet file
passengers_df.write.parquet("passengers.parquet")
